# Assignment Final
### INFO-615

## Trucks and Barge Route



In [2]:
# Imported Packages
import numpy as np
import pandas as pd
import geopy.distance
import seaborn as sns
import geopandas as gpd
from geopandas import GeoSeries, GeoSeries
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2



## Travelling Salesman problem

### Importing Data

In [4]:
hpo_small= pd.read_csv("../../GIS Final Assignment/Part 1- Data Cleaning and Basic Assessment/hp_orange_small.csv")

#### For Small Farms Only 

In [5]:
# Adding a new row for the zero address( port address) 
new_row = pd.DataFrame({'Unname':0, 'Business Name':'Port - City of Newburgh','Full Address':'2 Front St, Newburgh, NY 12550', 
                        'Latitude': 41.5010723, 'Longitude': -74.0076987,'RegisteredAcre':0 , 'index_right':35, 'NAME': 'Orange'},
                                                            index =[0])
    
# simply concatenate both dataframes
hpo_small = pd.concat([new_row, hpo_small]).reset_index(drop = True)
 



In [6]:

hpo_small = hpo_small.assign(ID=range(len(hpo_small)))

hpo_small

,Unname,Business Name,Full Address,Latitude,Longitude,RegisteredAcre,index_right,NAME,Unnamed: 0,RegisteredAcres,geometry,ID
0,0.0,Port - City of Newburgh,"2 Front St, Newburgh, NY 12550",41.501072,-74.007699,0.0,35,Orange,NaN,NaN,NaN,0
1,NaN,Big M Direct LLC,PO Box 311 Pine Island NY 10969,41.297872,-74.459324,NaN,35,Orange,53.0,8.0,POINT (-74.459324 41.2978722),1
2,NaN,Chester Agricultural Center LLC,PO Box 269 Mountainville NY 10918,41.404068,-74.078050,NaN,35,Orange,112.0,4.0,POINT (-74.0780502 41.4040683),2
3,NaN,Chickidoo Hempstead,199 Tally Ho Road Middletown NY 10940,41.454206,-74.504500,NaN,35,Orange,113.0,0.5,POINT (-74.5045003 41.4542061),3
4,NaN,Denice Davis LLC,158 Pulaski Highway Pine Island NY 13408,41.302306,-74.449586,NaN,35,Orange,156.0,1.0,POINT (-74.4495859 41.3023059),4
5,NaN,Green Harvest Farms LLC,64A Upper Wisner Rd Warwick NY 10990,41.263815,-74.325382,NaN,35,Orange,270.0,2.0,POINT (-74.3253823 41.2638147),5
6,NaN,Gullybean Inc.,PO Box 269 Mountainville NY 10953,41.389303,-74.093897,NaN,35,Orange,284.0,10.0,POINT (-74.0938969 41.389303),6
7,NaN,GWM Distribution LLC,"320 Robinson Avenue, Suite 212 Newburgh NY 12550",41.513458,-74.019777,NaN,35,Orange,286.0,4.0,POINT (-74.0197768 41.5134577),7
8,NaN,Joe Sidoti,10 Hillcrest Drive Westtown NY 10998,41.320805,-74.563423,NaN,35,Orange,388.0,20.0,POINT (-74.5634231 41.3208053),8
9,NaN,Kanna Worx Industries LLC,49 Mountain View Terrace Walden NY 12586,41.545423,-74.126038,NaN,35,Orange,403.0,6.0,POINT (-74.1260376 41.5454228),9


In [7]:
coord_data= hpo_small[["Latitude", "Longitude"]].values.tolist()
ID= hpo_small["ID"].astype(int).values.tolist()


In [8]:
#### Creating a distance Matrix

In [9]:
from geopy import distance
import pandas as pd

def make_distance_matrix(coords, addresses):
    from_list = []
    to_list = []
    distance_list = []

    for a in list(range(len(coords))):
        for b in list(range(len(coords))):

            origin_coord = coords[a]
            dest_coord = coords[b]

            origin_name = addresses[a]
            dest_name = addresses[b]

            dist = distance.distance(origin_coord, dest_coord).miles

            from_list.append(origin_name)
            to_list.append(dest_name)
            distance_list.append(dist)

    # Create a matrix containing the results
    distance_matrix = pd.DataFrame()
    distance_matrix['from'] = from_list
    distance_matrix['to'] = to_list
    distance_matrix['distance'] = distance_list

    # Do a pivot table
    distance_matrix = distance_matrix.pivot_table(values='distance', index='from', columns='to')
    
    return distance_matrix

In [10]:
dm_s= make_distance_matrix(coord_data, ID)

In [11]:
dm_s

to,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
from,,,,,,,,,,,,,,,,,,,,,
0,0.000000,27.337923,7.626023,25.986005,26.745940,23.253439,8.917906,1.059795,31.437677,6.858322,...,13.053710,23.733762,26.162769,26.366484,19.456590,26.047146,10.307549,30.380207,26.991532,10.969580
1,27.337923,0.000000,21.137647,11.041251,0.591984,7.358010,20.024693,27.256242,5.643152,24.322204,...,14.364062,3.618487,6.593844,5.135391,20.736055,1.577674,17.354028,3.079244,0.847748,23.582383
2,7.626023,21.137647,0.000000,22.417838,20.556274,16.099193,1.310105,8.132715,25.881251,10.068009,...,8.120208,17.674193,18.985963,19.368689,19.598700,20.047696,6.904202,24.216276,20.656545,13.288801
3,25.986005,11.041251,22.417838,0.000000,10.864345,16.104827,21.793794,25.484879,9.702006,20.619876,...,14.308609,10.156072,16.832552,15.530229,11.853255,9.829432,16.067095,11.935879,11.669093,18.123682
4,26.745940,0.591984,20.556274,10.864345,0.000000,6.989783,19.447481,26.664666,6.059218,23.747238,...,13.773623,3.028818,6.445250,5.003360,20.322461,1.136373,16.766142,3.664432,0.804902,23.039891
5,23.253439,7.358010,16.099193,16.104827,6.989783,0.000000,14.832462,23.431978,12.998673,22.021730,...,11.640901,6.016118,2.909332,3.400200,22.575659,7.354863,14.620071,9.910698,6.544913,22.558420
6,8.917906,20.024693,1.310105,21.793794,19.447481,14.832462,0.000000,9.392636,24.867623,10.902441,...,7.502144,16.604210,17.711845,18.121367,19.686910,18.976619,6.786897,23.099902,19.520773,13.873693
7,1.059795,27.256242,8.132715,25.484879,26.664666,23.431978,9.392636,0.000000,31.214312,5.935685,...,12.909775,23.641832,26.338578,26.484474,18.602071,25.930547,10.050138,30.279026,26.940274,10.025764
8,31.437677,5.643152,25.881251,9.702006,6.059218,12.998673,24.867623,31.214312,0.000000,27.500688,...,18.414450,8.368741,11.936810,10.493860,21.247708,6.080778,21.164186,3.731713,6.455763,25.994582


#### To Ensure that the distance matrix has only integers and large number so it cannot be rounded off to zero

In [12]:

dm_s= dm_s*100

dm_s_f= dm_s.astype(int)

### Converting Distance Matrix to array 

In [13]:
distarray_s= dm_s_f.to_numpy()


In [14]:
distarray_s

array([[   0, 2733,  762, 2598, 2674, 2325,  891,  105, 3143,  685, 2884,
        2577, 1305, 2373, 2616, 2636, 1945, 2604, 1030, 3038, 2699, 1096],
       [2733,    0, 2113, 1104,   59,  735, 2002, 2725,  564, 2432,  160,
         170, 1436,  361,  659,  513, 2073,  157, 1735,  307,   84, 2358],
       [ 762, 2113,    0, 2241, 2055, 1609,  131,  813, 2588, 1006, 2252,
        1974,  812, 1767, 1898, 1936, 1959, 2004,  690, 2421, 2065, 1328],
       [2598, 1104, 2241,    0, 1086, 1610, 2179, 2548,  970, 2061, 1211,
         998, 1430, 1015, 1683, 1553, 1185,  982, 1606, 1193, 1166, 1812],
       [2674,   59, 2055, 1086,    0,  698, 1944, 2666,  605, 2374,  216,
         118, 1377,  302,  644,  500, 2032,  113, 1676,  366,   80, 2303],
       [2325,  735, 1609, 1610,  698,    0, 1483, 2343, 1299, 2202,  804,
         703, 1164,  601,  290,  340, 2257,  735, 1462,  991,  654, 2255],
       [ 891, 2002,  131, 2179, 1944, 1483,    0,  939, 2486, 1090, 2138,
        1866,  750, 1660, 1771, 

### Defining Data for the Travelling Saleperson Problem

In [15]:
def create_data_model(dma):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = dma
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

In [16]:
data = create_data_model(distarray_s)
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

In [17]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  

In [18]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [19]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [20]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

In [21]:
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

Objective: 9427 miles
Route for vehicle 0:
 0 -> 7 -> 9 -> 21 -> 16 -> 3 -> 8 -> 19 -> 10 -> 1 -> 20 -> 4 -> 17 -> 11 -> 13 -> 15 -> 14 -> 5 -> 12 -> 18 -> 6 -> 2 -> 0

